# 助教哥你好呀~

### 这个文件是可以**直接运行**的加载模型的代码

-----

*因为中间有一些处理数据的过程, 整个文件运行时间大概在十分钟, 用jupyter notebook打开可以看到我最后一次运行与输出的结果*

##### 助教哥辛苦了

In [1]:
from keras.models import load_model

model = load_model('最高分的训练好的模型.h5')

E:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


**接下来 读取预处理好的测试数据**

In [15]:
import pandas as pd
import jieba

dff = pd.read_csv("./Preprocessed_data/train.csv",index_col=0)
dff['text'] = dff['text'].fillna('')
dff.head(10)

,text,class,positive
index,,,
0,﻿18年结婚 哈哈哈,0,0.900696
1,2017最后顿大餐吃完两人世界明年就是三个人一起啦许下生日愿望️希望一家人都能顺利平安健康🏻🏻🏻,1,0.999904
2,意盎然的季节！祝愿大家都生机勃勃，郁郁葱葱！,2,0.736431
3,2017 遇见挚友 遇见我老公 结了婚有了小芒果 希望2018也超级美好️,3,0.983905
4,2018.1.1,4,0.500000
5,2018加油！,5,0.895319
6,2018年做一个更加真实的自己。️,3,0.783433
7,2018年的第一天，完美的错过了一辆公交车。 德州,6,0.934181
8,2018年目标1.赚钱买房2.谈场恋爱，遇到对的人就结婚3.拥有一副健康的身体4.学会一种乐...,7,0.999799


In [16]:
dfTest = pd.read_csv("./Preprocessed_data/test.csv",index_col=0)
dfTest['text'] = dfTest['text'].fillna('')
dfTest.head(10)

,text,class,positive
index,,,
0,我是正面哦,0,0.347826
1,爱是恒久忍耐，又有恩慈。爱是不嫉妒，不自夸，不张狂，不轻易发怒。不计算人的恶。凡事包容。凡事...,0,0.496333
2,讨厌死了，上班上班上班不停的上班我真的超级累。什么都不干还是超级超级累。,0,0.000422
3,矮马大半夜的放肌肉男不让人睡觉了,0,0.409895
4,谢谢陈先生。,0,0.768959
5,我的2016要早点睡别熬夜,0,0.625607
6,周锐锐哥！爱你,0,0.970187
7,塞尼亚岛,0,0.500000
8,只可惜没能去现场,0,0.100791


**还有一点处理, 很快了**

In [17]:
def stopwordslist():
    f = open("./Preprocessed_data/stop.txt", "r")
    line = f.readline()
    stopwords = []
    index = 0
    while line:
        index += 1
        line = line.replace('\n', '')
        line = line.replace('[', '')
        line = line.replace(']', '')
        line = line.replace('］', '')
        line = line.replace('［', '')
        
        stopwords.append(line)
        line = f.readline()

    return stopwords

stopwords = stopwordslist()

def seg_depart(sentence):
    sentence_depart = jieba.cut(sentence.strip())
    outstr = ''
    for word in sentence_depart:
        if word not in stopwords:
            if word != '\t':
                outstr += word
                outstr += " "
    return outstr

sen = dff['text'].values

for i in range(len(sen)):
    if i % 100000 == 0:
        print(i)
    sen[i] = seg_depart(sen[i])
    

senTest = dfTest['text'].values

for i in range(len(senTest)):
    if i % 100000 == 0:
        print(i)
    senTest[i] = seg_depart(senTest[i])
    

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

MAX_NB_WORDS = 20000
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS, char_level=False)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Kai\AppData\Local\Temp\jieba.cache


0


Loading model cost 0.810 seconds.
Prefix dict has been built succesfully.


100000
200000
300000
400000
500000
600000
700000
800000
0
100000


E:\Anaconda3\lib\site-packages\keras_preprocessing\text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


**上面的输出是运行进度的一些信息, 上面的cell大概需要运行五分钟**

*很快就好啦*

In [18]:
tokenizer.fit_on_texts(sen)
sequences_test = tokenizer.texts_to_sequences(senTest)
MAX_SEQUENCE_LENGTH = 300

x_test = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH)

In [22]:
import numpy as np
import csv

pred = model.predict(x_test)
result = np.argmax(pred, axis = 1)

# 写入文件
csvFile = open('FORCheckResult.csv','w', newline='', encoding='UTF-8') # 设置newline，否则两行之间会空一行
writer = csv.writer(csvFile)

writer.writerow(['ID', 'Expected'])
for i in range(len(result)):
    if i % 50000 == 0:
        print(i)
    writer.writerow([int(i), int(result[i])])
    
csvFile.close()

0
50000
100000
150000


## 最高分的训练好的模型.h5  预测的 test.data 已经被输出到当前文件夹下的 FORCheckResult.csv 啦



#### 辛苦了 {心}